<a href="https://colab.research.google.com/github/rizkiar00/Fuzzy-Logic/blob/master/NaiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## IMPORT

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt; 
from google.colab import files
from sklearn.model_selection import train_test_split
import math
import operator 
%matplotlib inline
!git clone https://github.com/rizkiar00/NaiveBayes

Cloning into 'NaiveBayes'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 21 (delta 7), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (21/21), done.


## PISAH TRAINING DAN TESTING

In [131]:
data = pd.read_csv('NaiveBayes/TrainsetTugas1ML.csv',skipinitialspace=True) #Masukkan Data dan Buat Data Training,Testing
datasmall = data.loc[data['income'] == '>50K'].reset_index()
databig = data.loc[data['income'] == '<=50K'].reset_index()
datatrain = pd.concat([datasmall.loc[40:],databig.loc[5:]], ignore_index=True).drop(["index"], axis=1)
datatest = pd.concat([datasmall.loc[0:39],databig.loc[0:0]], ignore_index=True).drop(["index"], axis=1)
datatrainbig = datatrain.loc[datatrain['income'] == '>50K'].reset_index().drop(["index"], axis=1)
datatrainsmall = datatrain.loc[datatrain['income'] == '<=50K'].reset_index().drop(["index"], axis=1)
datatrainsmall

,id,age,workclass,education,marital-status,occupation,relationship,hours-per-week,income
0,30039,young,Private,HS-grad,Never-married,Craft-repair,Own-child,normal,<=50K
1,23050,adult,Self-emp-not-inc,Bachelors,Never-married,Exec-managerial,Own-child,normal,<=50K
2,27672,adult,Private,Bachelors,Never-married,Craft-repair,Not-in-family,normal,<=50K
3,21118,young,Private,HS-grad,Never-married,Craft-repair,Own-child,low,<=50K
4,25440,adult,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,normal,<=50K
5,27809,young,Private,Some-college,Married-civ-spouse,Exec-managerial,Husband,many,<=50K
6,12177,adult,Private,HS-grad,Divorced,Exec-managerial,Not-in-family,normal,<=50K
7,5956,adult,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,normal,<=50K
8,12742,young,Private,HS-grad,Never-married,Craft-repair,Not-in-family,normal,<=50K
9,17170,young,Private,Bachelors,Never-married,Exec-managerial,Not-in-family,low,<=50K


## HITUNG PRIOR & LIKELIHOOD

In [132]:
totalbig = datatrainbig['workclass'].count()+3 #Tambah satu buat di laplace smoothing
totalsmall = datatrainsmall['workclass'].count()+3
d = {'big':[totalbig],'small':[totalsmall]}
totalbigsmall = pd.DataFrame(data=d)

youngbig = (datatrainbig['age'][datatrainbig['age'] == 'young'].count()+1)
adultbig = (datatrainbig['age'][datatrainbig['age'] == 'adult'].count()+1)
oldbig = (datatrainbig['age'][datatrainbig['age'] == 'old'].count()+1)
youngsmall = (datatrainsmall['age'][datatrainsmall['age'] == 'young'].count()+1)
adultsmall = (datatrainsmall['age'][datatrainsmall['age'] == 'adult'].count()+1)
oldsmall = (datatrainsmall['age'][datatrainsmall['age'] == 'old'].count()+1)
d = {'': ["young", "adult","old"], 'big': [youngbig, adultbig,oldbig], 'small': [youngsmall , adultsmall, oldsmall]}
umur = pd.DataFrame(data=d)

Privatebig = (datatrainbig['workclass'][datatrainbig['workclass'] == 'Private'].count()+1)
Selfbig = (datatrainbig['workclass'][datatrainbig['workclass'] == 'Self-emp-not-inc'].count()+1)
Localbig = (datatrainbig['workclass'][datatrainbig['workclass'] == 'Local-gov'].count()+1)
Privatesmall = (datatrainsmall['workclass'][datatrainsmall['workclass'] == 'Private'].count()+1)
Selfsmall = (datatrainsmall['workclass'][datatrainsmall['workclass'] == 'Self-emp-not-inc'].count()+1)
Localsmall = (datatrainsmall['workclass'][datatrainsmall['workclass'] == 'Local-gov'].count()+1)
d = {'': ["Private", "Self-emp-not-inc","Local-gov"], 'big': [Privatebig, Selfbig,Localbig], 'small': [Privatesmall , Selfsmall, Localsmall]}
workclass = pd.DataFrame(data=d)

Somebig = (datatrainbig['education'][datatrainbig['education'] == 'Some-college'].count()+1)
Bachelorsbig = (datatrainbig['education'][datatrainbig['education'] == 'Bachelors'].count()+1)
HSbig = (datatrainbig['education'][datatrainbig['education'] == 'HS-grad'].count()+1)
Somesmall = (datatrainsmall['education'][datatrainsmall['education'] == 'Some-college'].count()+1)
Bachelorssmall = (datatrainsmall['education'][datatrainsmall['education'] == 'Bachelors'].count()+1)
HSsmall = (datatrainsmall['education'][datatrainsmall['education'] == 'HS-grad'].count()+1)
d = {'': ["Some-college", "Bachelors","HS-grad"], 'big': [Somebig, Bachelorsbig,HSbig], 'small': [Somesmall , Bachelorssmall, HSsmall]}
education = pd.DataFrame(data=d)

Marriedbig = (datatrainbig['marital-status'][datatrainbig['marital-status'] == 'Married-civ-spouse'].count()+1)
Divorcedbig = (datatrainbig['marital-status'][datatrainbig['marital-status'] == 'Divorced'].count()+1)
Neverbig = (datatrainbig['marital-status'][datatrainbig['marital-status'] == 'Never-married'].count()+1)
Marriedsmall = (datatrainsmall['marital-status'][datatrainsmall['marital-status'] == 'Married-civ-spouse'].count()+1)
Divorcedsmall = (datatrainsmall['marital-status'][datatrainsmall['marital-status'] == 'Divorced'].count()+1)
Neversmall = (datatrainsmall['marital-status'][datatrainsmall['marital-status'] == 'Never-married'].count()+1)
d = {'': ["Married-civ-spouse", "Divorced","Never-married"], 'big': [Somebig, Bachelorsbig,HSbig], 'small': [Somesmall , Bachelorssmall, HSsmall]}
marital = pd.DataFrame(data=d)

Craftbig = (datatrainbig['occupation'][datatrainbig['occupation'] == 'Craft-repair'].count()+1)
Execbig = (datatrainbig['occupation'][datatrainbig['occupation'] == 'Exec-managerial'].count()+1)
Profbig = (datatrainbig['occupation'][datatrainbig['occupation'] == 'Prof-specialty'].count()+1)
Craftsmall = (datatrainsmall['occupation'][datatrainsmall['occupation'] == 'Craft-repair'].count()+1)
Execsmall = (datatrainsmall['occupation'][datatrainsmall['occupation'] == 'Exec-managerial'].count()+1)
Profsmall = (datatrainsmall['occupation'][datatrainsmall['occupation'] == 'Prof-specialty'].count()+1)
d = {'': ["Craft-repair", "Exec-managerial","Prof-specialty"], 'big': [Craftbig, Execbig,Profbig], 'small': [Craftsmall , Execsmall, Profsmall]}
occupation = pd.DataFrame(data=d)

Husbandbig = (datatrainbig['relationship'][datatrainbig['relationship'] == 'Husband'].count()+1)
Notbig = (datatrainbig['relationship'][datatrainbig['relationship'] == 'Not-in-family'].count()+1)
Ownbig = (datatrainbig['relationship'][datatrainbig['relationship'] == 'Own-child'].count()+1)
Husbandsmall = (datatrainsmall['relationship'][datatrainsmall['relationship'] == 'Husband'].count()+1)
Notsmall = (datatrainsmall['relationship'][datatrainsmall['relationship'] == 'Not-in-family'].count()+1)
Ownsmall = (datatrainsmall['relationship'][datatrainsmall['relationship'] == 'Own-child'].count()+1)
d = {'': ["Husband", "Not-in-family","Own-child"], 'big': [Husbandbig, Notbig,Ownbig], 'small': [Husbandsmall , Notsmall, Ownsmall]}
relationship = pd.DataFrame(data=d)

normalbig = (datatrainbig['hours-per-week'][datatrainbig['hours-per-week'] == 'normal'].count()+1)
lowbig = (datatrainbig['hours-per-week'][datatrainbig['hours-per-week'] == 'low'].count()+1)
manybig = (datatrainbig['hours-per-week'][datatrainbig['hours-per-week'] == 'many'].count()+1)
normalsmall = (datatrainsmall['hours-per-week'][datatrainsmall['hours-per-week'] == 'normal'].count()+1)
lowsmall = (datatrainsmall['hours-per-week'][datatrainsmall['hours-per-week'] == 'low'].count()+1)
manysmall = (datatrainsmall['hours-per-week'][datatrainsmall['hours-per-week'] == 'many'].count()+1)
d = {'': ["normal", "low","many"], 'big': [normalbig, lowbig,manybig], 'small': [normalsmall , lowsmall,manysmall]}
hours = pd.DataFrame(data=d)

# normal = datatrain['hours-per-week'][data['hours-per-week'] == 'normal'].count()/total
# low = datatrain['hours-per-week'][data['hours-per-week'] == 'low'].count()/total

# lebih = datatrain['income'][data['income'] == '<=50K'].count()
# kurang = datatrain['income'][data['income'] == '>50K'].count()

# data_means = np.var([lebih,kurang])

print(umur)
print() 
print(workclass)
print() 
print(education)
print() 
print(marital)
print() 
print(occupation)
print() 
print(relationship)
print() 
print(hours) 
print() 
print(totalbigsmall) 

          big  small
0  young   44     20
1  adult   37     16
2    old    2      2

                     big  small
0           Private   72     29
1  Self-emp-not-inc    4      7
2         Local-gov    7      2

                 big  small
0  Some-college   20     14
1     Bachelors   45      8
2       HS-grad   18     16

                       big  small
0  Married-civ-spouse   20     14
1            Divorced   45      8
2       Never-married   18     16

                    big  small
0     Craft-repair   26     17
1  Exec-managerial   32     16
2   Prof-specialty   25      5

                  big  small
0        Husband   76     17
1  Not-in-family    5     13
2      Own-child    2      8

           big  small
0  normal   77     29
1     low    4      6
2    many    2      3

   big  small
0   83     38


In [0]:
def prediksi(datatest):
  hasiltest = []
  hasilkecil = []
  hasilbesar = []
  nilaibesar = 1
  nilaikecil = 1
  for i, row1 in datatest.iterrows():
    for j, row2 in umur.iterrows():
      if row1['age'] == row2['']:
        nilaibesar *= row2['big']
        nilaikecil *= row2['small']
    for j, row2 in workclass.iterrows():
      if row1['workclass'] == row2['']:
        nilaibesar *= row2['big']
        nilaikecil *= row2['small']
    for j, row2 in education.iterrows():
      if row1['education'] == row2['']:
        nilaibesar *= row2['big']
        nilaikecil *= row2['small']
    for j, row2 in marital.iterrows():
      if row1['marital-status'] == row2['']:
        nilaibesar *= row2['big']
        nilaikecil *= row2['small']
    for j, row2 in occupation.iterrows():
      if row1['occupation'] == row2['']:
        nilaibesar *= row2['big']
        nilaikecil *= row2['small']
    for j, row2 in relationship.iterrows():
      if row1['relationship'] == row2['']:
        nilaibesar *= row2['big']
        nilaikecil *= row2['small']
    for j, row2 in hours.iterrows():
      if row1['hours-per-week'] == row2['']:
        nilaibesar *= row2['big']
        nilaikecil *= row2['small']
    totaldata = totalbig+totalsmall
    nilaibesar *= totalbig/((totalbig**7)*totaldata)
    nilaikecil *= totalsmall/((totalsmall**7)*totaldata)
    hasilkecil.append(nilaikecil)
    hasilbesar.append(nilaibesar)
    if nilaibesar > nilaikecil:
      hasiltest.append('>50K')
    else:
      hasiltest.append('<=50K')
  return hasiltest

In [135]:
prediksi(datatest)

['>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K',
 '>50K']

In [136]:
datatest

,id,age,workclass,education,marital-status,occupation,relationship,hours-per-week,income
0,4776,young,Private,Some-college,Married-civ-spouse,Prof-specialty,Husband,normal,>50K
1,8173,adult,Private,Bachelors,Never-married,Prof-specialty,Not-in-family,normal,>50K
2,23423,young,Private,Some-college,Married-civ-spouse,Prof-specialty,Husband,normal,>50K
3,14894,adult,Private,Bachelors,Divorced,Exec-managerial,Not-in-family,normal,>50K
4,13128,adult,Local-gov,Bachelors,Married-civ-spouse,Prof-specialty,Husband,normal,>50K
5,29619,adult,Private,Some-college,Married-civ-spouse,Prof-specialty,Husband,normal,>50K
6,2308,adult,Private,Bachelors,Married-civ-spouse,Prof-specialty,Husband,normal,>50K
7,15394,adult,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,normal,>50K
8,14741,young,Private,HS-grad,Married-civ-spouse,Exec-managerial,Husband,normal,>50K
9,21019,adult,Local-gov,HS-grad,Married-civ-spouse,Craft-repair,Husband,normal,>50K


In [137]:
predict = pd.read_csv('NaiveBayes/TestsetTugas1ML.csv',skipinitialspace=True)
print(prediksi(predict))

['<=50K', '<=50K', '>50K', '>50K', '>50K', '>50K', '>50K', '<=50K', '<=50K', '>50K', '>50K', '>50K', '>50K', '>50K', '>50K', '>50K', '>50K', '>50K', '>50K', '>50K', '>50K', '>50K', '>50K', '>50K', '>50K', '>50K', '>50K', '>50K', '>50K', '>50K', '>50K', '>50K', '>50K', '>50K', '>50K', '>50K', '>50K', '>50K', '>50K', '>50K']
